# Import libraries

In [8]:
%run ./import_functions.ipynb

In [2]:
%run ./import_libraries.ipynb

Using TensorFlow backend.
/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/lekhag/opt/anaconda3/lib/python3.7

In [3]:
%run ./import_global_params.ipynb

# Real Data

In [4]:
# date_label=datetime.now().strftime("_%m_%d_%Y")
date_label='_2_4_2021'
newpath = os.getcwd()+'/'+date_label
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
data_path='/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version/'
    
data_source_ad=data_path+'/Data/AD/'
data_source_large=data_path+'/Data/Stream/Large'
data_source_covid=os.getcwd()+'/COVID-19/csse_covid_19_data/csse_covid_19_time_series/'


if not os.path.exists(data_source_large):
    os.makedirs(data_source_large)

results_path_non_stream_ad=newpath+'/Results/non_streaming/AD/'
results_path_non_stream_clust=newpath+'/Results/non_streaming/Clustering/'
results_path_covid=newpath+'/Results/COVID/'
results_path_non_stream_ad_large=newpath+'/Results/non_streaming/Large/AD/'
results_path_non_stream_clust_large=newpath+'/Results/non_streaming/Large/Clustering/'



image_path_non_stream_ad=results_path_non_stream_ad+'images/'
image_path_non_stream_clust=results_path_non_stream_clust+'images/'
image_path_covid=results_path_covid+'images/'
image_path_non_stream_ad_large=results_path_non_stream_ad_large+'images/'
image_path_non_stream_clust_large=results_path_non_stream_clust_large+'images/'



if not os.path.exists(image_path_non_stream_ad):
    os.makedirs(image_path_non_stream_ad)
if not os.path.exists(image_path_non_stream_clust):
    os.makedirs(image_path_non_stream_clust)
if not os.path.exists(image_path_covid):
    os.makedirs(image_path_covid)    
if not os.path.exists(image_path_non_stream_ad_large):
    os.makedirs(image_path_non_stream_ad_large)
if not os.path.exists(image_path_non_stream_clust_large):
    os.makedirs(image_path_non_stream_clust_large)




###### Model results

In [5]:
from scipy.io import arff
def load_data(file_path):
    filename, extension = os.path.splitext(file_path)
    name=(os.path.basename(file_path))
    if (extension=='.mat'):
        try:
            mat = scipy.io.loadmat(file_path)
            df = pd.DataFrame(np.hstack((mat['X'], mat['y'])))
            X,y=mat['X'], mat['y']
            return df,X,y

        except:
            try:
                arrays = {}
                f = h5py.File(file_path)
                for k, v in f.items():
                    arrays[k] = np.array(v)
                X,y=arrays['X'].T,arrays['y'].T
                df = pd.DataFrame(np.hstack((arrays['X'].T, arrays['y'].T)))
                return df,X,y
            except:
                print("1 Failed to load", name)

    elif extension=='.csv':
        try:
            df = pd.read_csv(file_path,low_memory=False,delimiter=',',header=None)
            df1=df[df.columns[(df.dtypes=='float')+(df.dtypes=='int')]]
            if df1.shape[1]==0:
                df = pd.read_csv(file_path,low_memory=False,delimiter=',')
                df1=df[df.columns[(df.dtypes=='float')+(df.dtypes=='int')]]                
            X=np.array(df1).astype(float)
            y=df.drop(df.columns[(df.dtypes=='float')+(df.dtypes=='int')],axis=1)
            return df,X,y
        except:
            try:
                df = pd.read_csv(file_path,low_memory=False,delimiter=',')
                df1=df[df.columns[(df.dtypes=='float')+(df.dtypes=='int')]]
                X=np.array(df1).astype(float)
                y=df.drop(df.columns[(df.dtypes=='float')+(df.dtypes=='int')],axis=1)
                return df,X,y
            except:
                print("2 Failed to load", name)

    elif extension=='.pickle':
        try:
            data=pickle.load(open(file_path,'rb'))
            X=data['X']
            y=data['y']
            return data,X,y
        except:
            try:
                d = pickle.load( open(file_path, "rb" ) )
                data=d['rawdata']
                labels=d['labels']
                X=np.array(data).astype(float)
                y=labels
                df=np.hstack((X,y))
                return df,X,y
            except:
                print("3 Failed to load", name)
                
    elif extension=='.arff':
        data = arff.loadarff(file_path)
        df = pd.DataFrame(data[0])
        df1=df[df.columns[(df.dtypes=='float')+(df.dtypes=='int')]]
        X=np.array(df1).astype(float)
        y=df.drop(df.columns[(df.dtypes=='float')+(df.dtypes=='int')],axis=1)
        return df,X,y
        
    else:
        print("Failed to load the extension",extension)
        pass

In [6]:
def ldp_cluster_kmeans_old(X,K_f,th_f,numiters,covariance_type,reg_covar,conv_criteria,cluster_train):
    thresholds=np.zeros(numiters)
    th=0.95
    N,D=X.shape
    if K_f>1:
        if D>30:
            pca = PCA(n_components=2).fit_transform(X)
            X_train, X_test = train_test_split(pca, train_size=cluster_train)
            kmeans = MiniBatchKMeans(n_clusters=K_f)
            kmeans = kmeans.partial_fit(X_train)
            z_1=kmeans.predict(pca)+1

        else:
            X_train, X_test = train_test_split(X, train_size=cluster_train)
            kmeans = MiniBatchKMeans(n_clusters=K_f)
            kmeans = kmeans.partial_fit(X_train)
            z_1=kmeans.predict(X)+1
    else:
        z_1=np.ones(N)        
        
    ana_score=np.zeros(N)
    converged_=0            

    for numiter in range(numiters):
        sys.stdout.write('*')
        clusters,sizes=np.unique(np.abs(z_1),return_counts=True)
        K=len(clusters)
        entropy_2 = np.empty((N, K, D))

        thetas=[]
        for k in clusters:
            ind_k_0=z_1==k
            ind_k=np.where(ind_k_0[ana_score<th])[0]
            c = len(ind_k)
            thetas.append(tuple((np.mean(X[ind_k],axis=0), 
                        np.array([np.cov(X[ind_k,d]) for d in range(D)]))))
        nk=sizes
        cc=range(len(thetas))
        means_=np.array([thetas[k][0].T for k in cc])
        covariances=np.array([thetas[k][1] for k in cc])
#         para_tuple=nk,means_,covariances


        n_samples, n_features = X.shape
        n_components, _ = means_.shape

        ss=[1/(cov) for cov in covariances]
        for k, (mu, prec_chol) in enumerate(zip(means_, ss)):
            prec_chol[np.isnan(prec_chol)]=0
            prec_chol[np.isinf(prec_chol)]=0
            y=(np.square(X-mu)*prec_chol)/2
            entropy_2[:, k, :] = (y)/sizes[k]
                
        entropy_DGProjection=np.max(entropy_2,axis=2)
        
        if K_f>1:
            cluster_log_probs=-(entropy_DGProjection*sizes)
#             cluster_prob=(np.exp(cluster_log_probs))
            z_1=np.argmax(cluster_log_probs,axis=1).flatten()+1


            

        ana_score=-np.max(-((entropy_DGProjection)), axis=1)
        ana_score=(ana_score-np.min(ana_score))/(np.max(ana_score)-np.min(ana_score))

#         ana_score=ana_score
        th=min(th_f,0.95,np.mean(ana_score)+2*np.cov(ana_score))

        ana_inds=ana_score>th

        z_1[ana_score>th]*=(-1)
        
        if np.sum(z_1<0)>min(0.25*N,D):
            z_1[np.argsort(-ana_score)[:int(0.25*N)]]=-1*np.abs(z_1[np.argsort(-ana_score)[:int(0.25*N)]])
            z_1[np.argsort(-ana_score)[int(0.25*N):]]=np.abs(z_1[np.argsort(-ana_score)[int(0.25*N):]])

        
        thresholds[numiter]=th
        
        if numiter > 100 and (np.max(np.abs(np.diff(thresholds[numiter-10:numiter])))<conv_criteria):
            converged_ += 1
            if converged_>0:
                print("Converged")
                break
                

    output={}
    output['z']=[z_1]
    output['entropy']=entropy_2
    output['prob']=ana_score
    output['thresholds']=thresholds
    return output

In [7]:
def ldp_cluster_kmeans(X,K_f,th_f,numiters,covariance_type,reg_covar,conv_criteria,cluster_train):
# if 1:
    thresholds=np.zeros(numiters)
    th=th_f
    N,D=X.shape
    if K_f>1:
        if D>20:
            pca = PCA(n_components=20).fit_transform(X)
            X_train, X_test = train_test_split(pca, train_size=cluster_train)
            kmeans = MiniBatchKMeans(n_clusters=K_f)
            kmeans = kmeans.partial_fit(X_train)
            z_1=kmeans.predict(pca)+1
#             clustering = OPTICS(min_samples=2).fit(pca)
#             z_1=clustering.labels_


        else:
            X_train, X_test = train_test_split(X, train_size=cluster_train)
            kmeans = MiniBatchKMeans(n_clusters=K_f)
            kmeans = kmeans.partial_fit(X_train)
            z_1=kmeans.predict(X)+1
    else:
        z_1=np.ones(N)        
        
    ana_score=np.zeros(N)
    converged_=0            

    for numiter in range(numiters):
        sys.stdout.write('*')
        clusters,sizes=np.unique(np.abs(z_1),return_counts=True)
        if len(clusters)<K_f:
            cluster_prob=(np.exp(cluster_log_probs))+sys.float_info.min
            cluster_prob/=np.sum(cluster_prob,axis=1)[:,np.newaxis]
            z_1=np.array([np.argmax(np.random.multinomial(1,cluster_prob[i]))+1 for i in range(N)])
            if len(np.unique(z_1))<K_f:
                K_f=len(np.unique(z_1))

#             break

            
        K=len(clusters)
        entropy_2 = np.empty((N, K, D))

        thetas=[]
        for k in clusters:
            ind_k_0=z_1==k
            ind_k=np.where(ind_k_0[ana_score<th])[0]
            c = len(ind_k)
            if c<3:
                ind_k=np.abs(z_1)==k
                thetas.append(tuple((np.mean(X[ind_k],axis=0), 
                            np.array([np.cov(X[:,d]) for d in range(D)]))))
            else:
                thetas.append(tuple((np.mean(X[ind_k],axis=0), 
                            np.array([np.cov(X[ind_k,d]) for d in range(D)]))))

                
        nk=sizes
        cc=range(len(thetas))
        means_=np.array([thetas[k][0].T for k in cc])
        covariances=np.array([thetas[k][1] for k in cc])
#         para_tuple=nk,means_,covariances


        n_samples, n_features = X.shape
        n_components, _ = means_.shape

        ss=[1/(cov) for cov in covariances]
        for k, (mu, prec_chol) in enumerate(zip(means_, ss)):
            prec_chol[np.isnan(prec_chol)]=0
            prec_chol[np.isinf(prec_chol)]=0
            y=(np.square(X-mu)*prec_chol)/2
            entropy_2[:, k, :] = (y)/sizes[k]
                
        entropy_DGProjection=np.max(entropy_2,axis=2)
        
        if K_f>1:
            cluster_log_probs=-(entropy_DGProjection*sizes)
#             cluster_prob=(np.exp(cluster_log_probs))
            z_1=np.argmax(cluster_log_probs,axis=1).flatten()+1

        ana_score=-np.max(-((entropy_DGProjection)), axis=1)
        ana_score=(ana_score-np.min(ana_score))/(np.max(ana_score)-np.min(ana_score))

#         ana_score=ana_score
        th=min(th_f,0.95,np.mean(ana_score)+2*np.cov(ana_score))

        ana_inds=ana_score>th

        z_1[ana_score>th]*=(-1)
        
        if np.sum(z_1<0)>min(0.25*N,D):
            z_1[np.argsort(-ana_score)[:int(0.25*N)]]=-1*np.abs(z_1[np.argsort(-ana_score)[:int(0.25*N)]])
            z_1[np.argsort(-ana_score)[int(0.25*N):]]=np.abs(z_1[np.argsort(-ana_score)[int(0.25*N):]])

        
        thresholds[numiter]=th
        
        if numiter > 100 and (np.max(np.abs(np.diff(thresholds[numiter-10:numiter])))<conv_criteria):
            converged_ += 1
            if converged_>0:
                print("Converged")
                break
                

    output={}
    output['z']=[z_1]
    output['entropy']=entropy_2
    output['prob']=ana_score
    output['thresholds']=thresholds
    return output

## Non-TS Data

###### Non Stream AD : Small datasets without clustering

In [8]:
# data_source=data_source_large
# results_path= results_path_non_stream_ad

data_source=data_source_ad
results_path= results_path_non_stream_ad



In [9]:
# Anomaly Detection Non-Stream
onlyfiles = [f for f in listdir(data_source) if isfile(join(data_source, f)) and f not in 
             ['.DS_Store','ecoli.mat'] ]
error_count=0
large_dataset_count=0
max_size=20001

for f in onlyfiles:
    file_path=data_source+'/'+f
    K=1
    th=0.95
    covariance_type='full'
    reg_covar=1e-16
    numiters=100
    conv_criteria=1e-30
    cluster_train=0.25


    try:
        df,X,y=load_data(file_path)
        XX=pd.DataFrame(X)
        print(f,X.shape)
        
        N,D=X.shape
        if N>max_size or D>30:
            large_dataset_count+=1
            shutil.move(file_path, data_source_large)

            
        else:
            start=time.time()
            output=ldp_cluster_kmeans(X,K,th,numiters,covariance_type,reg_covar,conv_criteria,cluster_train)
            end=time.time()
            output['file_path']=file_path
            output['comp_time_secs']=end-start

            pickle.dump( output, open(results_path+f+".pickle", "wb" ))

    except Exception as e:
        error_count+=1
        print(f)
        print(e)
print("large_dataset_count,error_count,len(onlyfiles)",large_dataset_count,error_count,len(onlyfiles))

/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  from ipykernel import kernelapp as app


http.mat (567498, 3)
http.mat
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version//Data/Stream/Large/http.mat' already exists
pima.mat (768, 8)
****************************************************************************************************wine.mat (129, 13)
****************************************************************************************************cardio.mat (1831, 21)
****************************************************************************************************mnist.mat (7603, 100)
mnist.mat
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version//Data/Stream/Large/mnist.mat' already exists
1 Failed to load arrhythmia.mat
arrhythmia.mat
cannot unpack non-iterable NoneType object
pendigits.mat (6870, 16)
****************************************************************************************************breast-cancer-unsupervised-ad.csv (367, 30)
*******

/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "


*********************************************************************************************shuttle.mat (49097, 9)
shuttle.mat
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version//Data/Stream/Large/shuttle.mat' already exists
1 Failed to load musk.mat
musk.mat
cannot unpack non-iterable NoneType object
smtp.pickle (113, 3)
****************************************************************************************************shuttle-unsupervised-ad.csv (46464, 9)
shuttle-unsupervised-ad.csv
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version//Data/Stream/Large/shuttle-unsupervised-ad.csv' already exists
thyroid.mat (3772, 6)
****************************************************************************************************seismic-bumps.arff (2584, 14)
************************************************************

/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: divide by zero encountered in true_divide


****************************************vowels.mat (1456, 12)
****************************************************************************************************1 Failed to load optdigits.mat
optdigits.mat
cannot unpack non-iterable NoneType object
breastw.mat (683, 9)
****************************************************************************************************1 Failed to load satellite.mat
satellite.mat
cannot unpack non-iterable NoneType object
lympho.mat (148, 18)
****************************************************************************************************1 Failed to load speech.mat
speech.mat
cannot unpack non-iterable NoneType object
annthyroid.mat (7200, 6)
****************************************************************************************************pen-global-unsupervised-ad.csv (809, 16)
****************************************************************************************************wbc.mat (378, 30)
******************************************************

###### Non Stream AD : Small datasets with clustering, K=2

In [10]:
data_source=data_source_ad
results_path= results_path_non_stream_clust

In [11]:
# Anomaly Detection Non-Stream
onlyfiles = [f for f in listdir(data_source) if isfile(join(data_source, f)) and f not in 
             ['.DS_Store','ecoli.mat'] ]
error_count=0
large_dataset_count=0
max_size=20001

for f in onlyfiles:
    file_path=data_source+'/'+f
    K=2
    th=0.95
    covariance_type='full'
    reg_covar=1e-16
    numiters=100
    conv_criteria=1e-30
    cluster_train=0.25


    try:
        df,X,y=load_data(file_path)
        XX=pd.DataFrame(X)
        print(f,X.shape)
        
        N,D=X.shape
        if N>max_size or D>30:
            large_dataset_count+=1
            shutil.move(file_path, data_source_large)
            
        else:
            start=time.time()
            output=ldp_cluster_kmeans(X,K,th,numiters,covariance_type,reg_covar,conv_criteria,cluster_train)
            end=time.time()
            output['file_path']=file_path
            output['comp_time_secs']=end-start

            pickle.dump( output, open(results_path+f+".pickle", "wb" ))

    except Exception as e:
        error_count+=1
        print(f)
        print(e)
print("large_dataset_count,error_count,len(onlyfiles)",large_dataset_count,error_count,len(onlyfiles))

http.mat (567498, 3)
http.mat
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version//Data/Stream/Large/http.mat' already exists
pima.mat (768, 8)
**************

/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  from ipykernel import kernelapp as app


**************************************************************************************wine.mat (129, 13)
****************************************************************************************************cardio.mat (1831, 21)
****************************************************************************************************mnist.mat (7603, 100)
mnist.mat
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version//Data/Stream/Large/mnist.mat' already exists
1 Failed to load arrhythmia.mat
arrhythmia.mat
cannot unpack non-iterable NoneType object
pendigits.mat (6870, 16)
******

/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  from ipykernel import kernelapp as app


**********************************************************************************************breast-cancer-unsupervised-ad.csv (367, 30)
********************************************************

...

******************************************wbc.mat (378, 30)
****************************************************************************************************cover.pickle (217, 10)
****************************************************************************************************http.pickle (110, 3)
****************************************************************************************************smtp.mat (95156, 3)
smtp.mat
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version//Data/Stream/Large/smtp.mat' already exists
mammography.mat (11183, 6)
**

/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: divide by zero encountered in true_divide
/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:83: RuntimeWarning: invalid value encountered in true_divide
/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  from ipykernel import kernelapp as app


**************************************************************************************************glass.mat (214, 9)
****************************************************************************************************1 Failed to load satimage-2.mat
satimage-2.mat
cannot unpack non-iterable NoneType object
cover.mat (286048, 10)
cover.mat
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version//Data/Stream/Large/cover.mat' already exists


/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: divide by zero encountered in true_divide
/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  from ipykernel import kernelapp as app
/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "


kdd99-unsupervised-ad.csv (620098, 29)
kdd99-unsupervised-ad.csv
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version//Data/Stream/Large/kdd99-unsupervised-ad.csv' already exists
vertebral.mat (240, 6)
****************************************************************************************************large_dataset_count,error_count,len(onlyfiles) 7 13 32


###### Non Stream AD : Large datasets without clustering and with K=2 clustering

In [12]:
data_source=data_source_large
results_path= results_path_non_stream_ad_large

# Anomaly Detection Non-Stream
onlyfiles = [f for f in listdir(data_source) if isfile(join(data_source, f)) and f not in 
             ['.DS_Store','ecoli.mat'] ]
error_count=0
large_dataset_count=0
max_size=20001

for f in onlyfiles:
    file_path=data_source+'/'+f
    K=1
    th=0.95
    covariance_type='full'
    reg_covar=1e-16
    numiters=100
    conv_criteria=1e-30
    cluster_train=0.25


    try:
        df,X,y=load_data(file_path)
        XX=pd.DataFrame(X)
        print(f,X.shape)
        
        N,D=X.shape
        start=time.time()
        output=ldp_cluster_kmeans(X,K,th,numiters,covariance_type,reg_covar,conv_criteria,cluster_train)
        end=time.time()
        output['file_path']=file_path
        output['comp_time_secs']=end-start

        pickle.dump( output, open(results_path+f+".pickle", "wb" ))

    except Exception as e:
        error_count+=1
        print(f)
        print(e)
print("large_dataset_count,error_count,len(onlyfiles)",large_dataset_count,error_count,len(onlyfiles))


data_source=data_source_large
results_path= results_path_non_stream_clust_large

# Anomaly Detection Non-Stream
onlyfiles = [f for f in listdir(data_source) if isfile(join(data_source, f)) and f not in 
             ['.DS_Store','ecoli.mat'] ]
error_count=0
large_dataset_count=0
max_size=20001

for f in onlyfiles:
    file_path=data_source+'/'+f
    K=2
    th=0.95
    covariance_type='full'
    reg_covar=1e-16
    numiters=100
    conv_criteria=1e-30
    cluster_train=0.25


    try:
        df,X,y=load_data(file_path)
        XX=pd.DataFrame(X)
        print(f,X.shape)
        
        N,D=X.shape
        start=time.time()
        output=ldp_cluster_kmeans(X,K,th,numiters,covariance_type,reg_covar,conv_criteria,cluster_train)
        end=time.time()
        output['file_path']=file_path
        output['comp_time_secs']=end-start

        pickle.dump( output, open(results_path+f+".pickle", "wb" ))

    except Exception as e:
        error_count+=1
        print(f)
        print(e)
print("large_dataset_count,error_count,len(onlyfiles)",large_dataset_count,error_count,len(onlyfiles))

/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  from ipykernel import kernelapp as app


http.mat (567498, 3)
****************************************************************************************************

/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "


skin.preproc.csv (245057, 4)
****************************************************************************************************mnist.mat (7603, 100)
*****

/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: divide by zero encountered in true_divide


...

opt.digits.preproc.csv (5620, 63)
************

/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: divide by zero encountered in true_divide


****************************************************************************************letter-unsupervised-ad.csv (1600, 32)


/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "


****************************************************************************************************large_dataset_count,error_count,len(onlyfiles) 0 0 29


###### Data Descriptions

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
def ana_label(y):
    labels,prop=np.unique(y, return_counts=True)
    return (y==labels[np.argmin(prop)])*1

In [15]:
data_source=data_source_ad

In [16]:
# Anomaly Detection Non-Stream
onlyfiles = [f for f in listdir(data_source) if isfile(join(data_source, f)) and f not in 
             ['.DS_Store','ecoli.mat'] ]
error_count=0
large_dataset_count=0
max_size=20001

for f in onlyfiles:
    file_path=data_source+'/'+f
    try:
        df,X,y=load_data(file_path)
        try:
            y=ana_label(y)
            if (y.shape[1])>1:
                try:
                    y=ana_label(y['class'])
                except:
                    try:
                        y=ana_label(y['ground.truth'])
                    except:
                        pass                
        except:
            try:
                y=ana_label(y['class'])
            except:
                try:
                    y=ana_label(y['ground.truth'])
                except:
                    pass
                
        if (y.shape[1])>1:
            print('issue')
            break
        
        print(f,' & ', X.shape[0], ' & ',X.shape[1], ' & ', (np.mean(y)*100).round(3),'\% \\\ ')
    except:
        pass
        


http.mat  &  567498  &  3  &  0.39 \% \\ 
pima.mat  &  768  &  8  &  34.896 \% \\ 
wine.mat  &  129  &  13  &  7.752 \% \\ 
cardio.mat  &  1831  &  21  &  9.612 \% \\ 
mnist.mat  &  7603  &  100  &  9.207 \% \\ 
1 Failed to load arrhythmia.mat
pendigits.mat  &  6870  &  16  &  2.271 \% \\ 
breast-cancer-unsupervised-ad.csv  &  367  &  30  &  30    2.725
dtype: float64 \% \\ 
shuttle.mat  &  49097  &  9  &  7.151 \% \\ 
1 Failed to load musk.mat
shuttle-unsupervised-ad.csv  &  46464  &  9  &  9    1.89
dtype: float64 \% \\ 
thyroid.mat  &  3772  &  6  &  2.466 \% \\ 
vowels.mat  &  1456  &  12  &  3.434 \% \\ 
1 Failed to load optdigits.mat
breastw.mat  &  683  &  9  &  34.993 \% \\ 
1 Failed to load satellite.mat
lympho.mat  &  148  &  18  &  4.054 \% \\ 
1 Failed to load speech.mat
annthyroid.mat  &  7200  &  6  &  7.417 \% \\ 
pen-global-unsupervised-ad.csv  &  809  &  16  &  16    11.125
dtype: float64 \% \\ 
wbc.mat  &  378  &  30  &  5.556 \% \\ 
smtp.mat  &  95156  &  3  &  0.032